In [1]:
#####################################################################
# Mandatory : Run this cell and restart the notebook kernel right after
#####################################################################
!pip install torch==1.11 transformers stanza

     |████████████████████████████████| 574 kB 22.8 MB/s eta 0:00:01
     |████████████████████████████████| 197 kB 23.2 MB/s eta 0:00:01
  Created wheel for emoji: filename=emoji-2.0.0-py3-none-any.whl size=193002 sha256=f2ba39c5bfc065aae279bed1af2dbc61228ec33cdc462937c87bff48a815fdbb
  Stored in directory: /home/pbeuran/.cache/pip/wheels/23/a5/a8/e74bad1ceced228b6ae94dcbacc5c67df6486fd1620714e7d1
Successfully built emoji


In [2]:
%cd /content

[Errno 2] No such file or directory: '/content'
/home/pbeuran/personal_repositories/deepqa/notebooks


In [ ]:
%%bash
#####################################################################
# Only use on Googgle Colab, uncomment if necessary
#####################################################################
# Clone the repo content into
cd /content
rm -rf deepqa
git clone -b model https://github.com/PaulBeuran/deepqa.git

In [ ]:
%%bash
#####################################################################
# Only use on Googgle Colab, uncomment if necessary
#####################################################################
# Clone the repo content into
cd /content
rm -rf deepqa
git clone -b model https://github.com/PaulBeuran/deepqa.git

In [15]:
#####################################################################
# Only use on Googgle Colab, uncomment if necessary
#####################################################################
%cd deepqa/notebooks/

[Errno 2] No such file or directory: 'deepqa/notebooks/'
/home/pbeuran/personal_repositories/deepqa/notebooks


In [4]:
%%bash
cd ..
curl -O https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
unzip -o glove.6B.zip -d word_encoders_configs
rm glove.6B.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  822M  100  822M    0     0  4124k      0  0:03:24  0:03:24 --:--:-- 4146k


Archive:  glove.6B.zip
  inflating: word_encoders_configs/glove.6B.50d.txt  
  inflating: word_encoders_configs/glove.6B.100d.txt  
  inflating: word_encoders_configs/glove.6B.200d.txt  
  inflating: word_encoders_configs/glove.6B.300d.txt  


In [2]:
%%bash
# Dowload the SQuAD1.1 data
curl -O https://data.deepai.org/squad1.1.zip
unzip -o squad1.1.zip -d data

2022-07-30 00:36:05.538503: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-30 00:36:05.538577: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [1]:
import sys
import os
import pathlib

import json

import pandas as pd
import numpy as np
import torch

from tqdm import tqdm

TORCH_CUDA_IS_AVAILABLE = torch.cuda.is_available()

if TORCH_CUDA_IS_AVAILABLE:
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

deepqa_lib_path = str(pathlib.Path(os.getcwd()).parent.parent.absolute())
sys.path.insert(0, deepqa_lib_path)

from deepqa import utils, preprocessing, tokenizer, module, model, wrapper, loss, metrics 

2022-08-06 14:18:17.022434: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-06 14:18:17.022510: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-08-06 14:18:19.649332: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-08-06 14:18:19.649555: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-06 14:18:19.649653: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcub

In [2]:
# Read train and dev data
with open("data/train-v1.1.json", "rb") as j:
    train_data = json.load(j)["data"]
with open("data/dev-v1.1.json", "rb") as j:
    dev_data = json.load(j)["data"]

In [3]:
train_tcqa = preprocessing.tabularize_squad11_data(train_data)
dev_tcqa = preprocessing.tabularize_squad11_data(dev_data)

In [4]:
glove_tokenizer = tokenizer.GLoVETokenizer(tokenize_char=True)

context_max_length = 300
query_max_length = 40
token_max_length = 40

train_tokens_cqa = glove_tokenizer.tokenize_qa_data(
    *(train_tcqa[1:]), context_max_length, query_max_length, token_max_length
)
dev_tokens_cqa = glove_tokenizer.tokenize_qa_data(
    *(dev_tcqa[1:]), context_max_length, query_max_length, token_max_length
)

train_dataset = utils.QADataset(*train_tokens_cqa, "cuda" if TORCH_CUDA_IS_AVAILABLE
                                                          else "cpu")
dev_dataset = utils.QADataset(*dev_tokens_cqa, "cuda" if TORCH_CUDA_IS_AVAILABLE
                                                      else "cpu")

In [5]:
char_cnn = module.CharCNN(1371, 16, [100], [5])

glove_word_encoder = module.GLoVEWordEncoder(embedding_size=200, char_encoder=char_cnn)
contextual_embedding_size = 100

In [6]:
bidaf = model.BiDAF(glove_word_encoder, contextual_embedding_size, 0.2)

In [7]:
train_loader = torch.utils.data.DataLoader(train_dataset, 64)
dev_loader = torch.utils.data.DataLoader(dev_dataset, 64)

In [ ]:
import numpy as np
import torch
from tqdm import tqdm 
from deepqa.metrics import exact_match, overlap_f1_score
from deepqa.loss import bi_cross_entropy

class QATrainWrapper(torch.nn.Module):

    def __init__(self, model, 
                 context_max_length=512, query_max_length=64, 
                 verbose=True):

        super(QATrainWrapper, self).__init__()
        self.model = model
        self.context_max_length = context_max_length
        self.query_max_length = query_max_length
        self.optimizer = torch.optim.Adam(model.parameters())
        self.verbose = verbose

    def train(self, train_data_iter, epochs, 
              loss=bi_cross_entropy, 
              metrics={"EM": exact_match, "F1": overlap_f1_score},
              val_data_iter=None):

        train_dataset_len = len(train_data_iter.dataset)
        train_batch_size = train_data_iter.batch_size
        train_loader_n_iter = len(train_data_iter)
        train_loss_by_epochs = np.zeros((epochs * train_dataset_len, 2))
        train_loss_by_epochs[:, 0] = np.repeat(np.arange(epochs), train_dataset_len)
        train_metrics_by_epochs = np.zeros((epochs * train_dataset_len, len(metrics) + 1))
        train_metrics_by_epochs[:, 0] = np.repeat(np.arange(epochs), train_dataset_len)
        if val_data_iter is not None:
            val_dataset_len = len(val_data_iter.dataset)
            val_batch_size = val_data_iter.batch_size
            val_loader_n_iter = len(val_data_iter)
            val_loss_by_epochs = np.zeros((epochs * val_dataset_len, 2))
            val_loss_by_epochs[:, 0] = np.repeat(np.arange(epochs), val_dataset_len)
            val_metrics_by_epochs = np.zeros((epochs * val_dataset_len, len(metrics) + 1))
            val_metrics_by_epochs[:, 0] = np.repeat(np.arange(epochs), val_dataset_len)
            
        for epoch in range(epochs):
            epoch_total_loss = 0
            epoch_total_metrics = {k:0 for k in metrics}
            data_iter_tqdm = tqdm(train_data_iter) if self.verbose else train_data_iter
            for i, batch in enumerate(data_iter_tqdm):
                contexts_tokens, queries_tokens, answers_tokens_range = tuple(batch.values())
                answers_probs = self.model([contexts_tokens, queries_tokens], True)
                answer_preds = answers_probs.argmax(dim=1)
                self.optimizer.zero_grad()
                loss_batch = loss(answers_probs, answers_tokens_range)
                loss_step = loss_batch.mean()
                loss_step.backward()
                self.optimizer.step()
                metrics_batch_dict = {k : metric(answer_preds, answers_tokens_range)
                                      for k,metric in metrics.items()}
                metrics_batch = torch.cat(list(metrics_batch_dict.values()), axis=1)            
                min_slice = (train_loader_n_iter * epoch) + (train_batch_size * i)
                max_slice = ((train_loader_n_iter * epoch) + (train_batch_size * i) +
                             train_batch_size if i+1 != train_loader_n_iter 
                                              else train_dataset_len % train_batch_size)
                batch_slice = slice(min_slice, max_slice)
                train_loss_by_epochs[batch_slice, 1] = loss_batch.to("cpu").numpy()
                train_metrics_by_epochs[batch_slice, 1:] = metrics_batch.to("cpu").numpy()
                epoch_total_loss += loss_step.to("cpu").item()
                epoch_total_metrics = {k : epoch_total_metrics[k] +
                                           metrics_batch_dict[k].mean().to("cpu").item()
                                       for k in metrics}
                if self.verbose:
                    data_iter_tqdm.set_description(f"""Epoch {epoch+1}/{epochs} - Train - Loss: {
                        round(epoch_total_loss/(i+1), 3)}, Metrics: {
                            {id:round(etm/(i+1), 3) for id, etm in epoch_total_metrics.items()}}""")

            if val_data_iter is not None:
                epoch_total_loss = 0
                epoch_total_metrics = {k:0 for k in metrics}
                data_iter_tqdm = tqdm(val_data_iter) if self.verbose else val_data_iter
                for i, batch in enumerate(data_iter_tqdm):
                    with torch.no_grad():
                        contexts_tokens, queries_tokens, answers_tokens_range = tuple(batch.values())
                        answers_probs = self.model([contexts_tokens, queries_tokens])
                        answer_preds = answers_probs.argmax(dim=1)
                        loss_batch = loss(answers_probs, answers_tokens_range)
                        loss_step = loss_batch.mean()
                        metrics_batch_dict = {k : metric(answer_preds, answers_tokens_range)
                                            for k,metric in metrics.items()}
                        metrics_batch = torch.cat(list(metrics_batch_dict.values()), axis=1)            
                        min_slice = (val_loader_n_iter * epoch) + (val_batch_size * i)
                        max_slice = ((val_loader_n_iter * epoch) + (val_batch_size * i) +
                                    val_batch_size if i+1 != val_loader_n_iter 
                                                    else val_dataset_len % val_batch_size)
                        batch_slice = slice(min_slice, max_slice)
                        val_loss_by_epochs[batch_slice, 1] = loss_batch.to("cpu").numpy()
                        val_metrics_by_epochs[batch_slice, 1:] = metrics_batch.to("cpu").numpy()
                        epoch_total_loss += loss_step.to("cpu").item()
                        epoch_total_metrics = {k : epoch_total_metrics[k] +
                                                metrics_batch_dict[k].mean().to("cpu").item()
                                            for k in metrics}
                        if self.verbose:
                            data_iter_tqdm.set_description(f"""Epoch {epoch+1}/{epochs} - Val - Loss: {
                                round(epoch_total_loss/(i+1), 3)}, Metrics: {
                                    {id:round(etm/(i+1), 3) for id, etm in epoch_total_metrics.items()}}""")

        to_return = [train_loss_by_epochs, train_metrics_by_epochs]
        if val_data_iter is not None:
            to_return.extend([val_loss_by_epochs, val_metrics_by_epochs])
        return to_return

In [8]:
bidaf_trainer = wrapper.QATrainWrapper(bidaf)
bidaf_trainer.train(train_loader, 
                    epochs=16, 
                    loss=loss.bi_cross_entropy,
                    val_data_iter=dev_loader)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [1]:
(10 + 5 + 7 + 8)/4

7.5

In [ ]:
x = [10, 5, 7, 8]
acc = 0
for i in range(4):
    acc = acc 